Kode for å fjerne duplikater på arbeidsuker. Duplikater har oppstått under henting av data.
Dette scriptet fjerner ekstra dag med samme vaktkode på angitt ukenummer.

In [ ]:
workweek = 34

In [ ]:
from lib.database.mysql import MySqlConnection

agent = MySqlConnection()

In [ ]:
# queries

all_employees_query = "select id from employee"
workdays_id_query = "select id from workday where weeknumber = %s"
shifts_query = """select s.id, s.shiftcode_id, s.timestamp from shift as s
	join employee as e on s.employee_id = e.id
    where e.id = %s and s.workday_id = %s
	order by s.timestamp asc;"""
delete_command = "delete from shift where id = %s"

In [ ]:
employees = agent.query_all(all_employees_query)
workdays = agent.query_all(workdays_id_query, (workweek,))

employees = [x[0] for x in employees]
workdays = [x[0] for x in workdays]


In [ ]:
s_id_to_delete = []

for emp in employees:
    for day in workdays:
        wd = agent.query_all(shifts_query, (emp, day))
        if not (len(wd) > 1): continue
        if wd[0][1] == wd[1][1]: s_id_to_delete.append(wd[1][0])

len(s_id_to_delete)

In [ ]:
s_id_to_delete = [(x,) for x in s_id_to_delete]
agent.command_many(delete_command, s_id_to_delete)

In [ ]:
# delete week 1 2025

to_delete = []
q = "select id from workday where weeknumber = 1 and year = 2025"
week1 = agent.query_all(q)
week1 = [x[0] for x in week1]

for emp in employees:
    for day in week1:
        wd = agent.query_all(shifts_query, (emp, day,))
        if not wd: continue
        to_delete.append(wd[0][0])

shifts_delete = [(x,) for x in to_delete]
workdays_delete = [(x,) for x in week1]
workday_delete_command = "delete from workday where id = %s"
agent.command_many(delete_command, to_delete )
agent.command_many(workday_delete_command, workdays_delete)